In [ ]:
import SciServer
from SciServer import Authentication, Files
from pprint import pprint
print('Modules imported')

# Define login Name and password before running these examples
Authentication_loginName = 'raddick'
#Authentication_loginPassword = 'moose4sciserver'

# Enter the SciServer login and password of someone you share files with (be careful to blank them out before sharing this notebook!!!)
Authentication_login_sharedWithName = '***'
Authentication_login_sharedWithPassword = '***'
print('User names and passwords set! BLANK THEM OUT BEFORE SHARING!')



# Get help

In [ ]:
#help(Files)

# Show the file services that are available to you

In [ ]:
# get available FileServices -- returns a list of dictionaries, one for each avialable file service
#   the for loop iterates through the list and prints information about each dictionary object

fileServicesList = Files.getFileServices()
for thisFileService in fileServicesList:
    print('name',':',thisFileService['name'])
    print('description',':',thisFileService['description'])
    print('apiEndpoint',':',thisFileService['apiEndpoint'])
    print('\n')
    print('The full dictionary:')
    pprint(thisFileService)

# Get file service information from a name

In [ ]:
# get FileService from Name
fileServiceNames = Files.getFileServicesNames()
print('Names of available file services:')
pprint(fileServiceNames)

print('\n')
Files_FileServiceName = "FileServiceJHU"
print('File Service information for {0:}:'.format(Files_FileServiceName))
fileService = Files.getFileServiceFromName(Files_FileServiceName)
pprint(fileService)

# Get API endpoint for a file service

In [ ]:
# get the API endpoint URL of a FileService
fileServiceAPIUrl = Files.__getFileServiceAPIUrl(fileService)
print(fileServiceAPIUrl)

# Show root volumes

In [ ]:
rootVolumes = Files.getRootVolumesInfo(fileService)
pprint(rootVolumes)

# Show user volumes

In [ ]:
userVolumes = Files.getUserVolumesInfo(fileService)
pprint(userVolumes)

# Create a new user volume

In [ ]:
Files_FileServiceName = "FileServiceJHU"
Files_RootVolumeName1 = 'Storage'
Files_VolumeName1 = "created_through_example_notebook"

fileService = Files.getFileServiceFromName(Files_FileServiceName)

Files.createUserVolume(fileService,"/".join([Files_RootVolumeName1,Authentication_loginName,Files_VolumeName1]))

print('Created new user volume: {0:}!'.format(Files_VolumeName1))

# Create a new directory in the persistent volume

In [ ]:
# create a directory in the persistent volume
Files_FileServiceName = "FileServiceJHU"
Files_RootVolumeName2 = "Storage"
Files_VolumeName2 = "persistent"
Files_NewDirectoryName2 = "directory_created_through_example_notebook"

fileService = Files.getFileServiceFromName(Files_FileServiceName)
Files.createDir(fileService, 
    "/".join([Files_RootVolumeName2,Authentication_loginName,Files_VolumeName2,Files_NewDirectoryName2]));

print('Created new user directory {0:} inside {1:} volume.'.format(Files_NewDirectoryName2, Files_VolumeName2))

# List contents of a directory

In [ ]:
# List content of remote directory
Files_FileServiceName = "FileServiceJHU"
Files_RootVolumeName3 = "Storage"
Files_VolumeName3 = "persistent"
Files_DirectoryName3 = ""

fileService = Files.getFileServiceFromName(Files_FileServiceName)

dirList = Files.dirList(fileService, 
    "/".join([Files_RootVolumeName3,Authentication_loginName,Files_VolumeName3,Files_DirectoryName3]),
    level=4)
pprint(dirList)

# Upload a file

In [ ]:
# upload a text string into a file in the remote directory
Files_FileServiceName = "FileServiceJHU"
Files_RootVolumeName4 = "Storage"
Files_VolumeName4 = "persistent"
Files_DirectoryName4 = "directory_created_through_example_notebook"

# This file gets created below, or replace with your own filename
Files_LocalFileName = "sharknado.csv"

# Create this simple file to upload
Files_LocalFileContent = "Year, Sharknado movies shown on U.S. television\n"
Files_LocalFileContent += "2012, 0\n"
Files_LocalFileContent += "2013, 1\n"
Files_LocalFileContent += "2014, 1\n"
Files_LocalFileContent += "2015, 1\n"
Files_LocalFileContent += "2016, 1\n"
Files_LocalFileContent += "2017, 1\n"

fileService = Files.getFileServiceFromName(Files_FileServiceName)

Files.upload(fileService, 
    "/".join([Files_RootVolumeName4,Authentication_loginName,Files_VolumeName4,Files_DirectoryName4,Files_LocalFileName]), 
             data=Files_LocalFileContent)

print('Uploaded file {0:} to {1:} volume in directory {2:}.'.format(Files_LocalFileName, Files_VolumeName4, Files_DirectoryName4))

# List contents of a directory

In [ ]:
# List content of remote directory
Files_FileServiceName = "FileServiceJHU"
Files_RootVolumeName3 = "Storage"
Files_VolumeName3 = "persistent"
Files_DirectoryName3 = "directory_created_through_example_notebook"

fileService = Files.getFileServiceFromName(Files_FileServiceName)

dirList = Files.dirList(fileService, 
    "/".join([Files_RootVolumeName3,Authentication_loginName,Files_VolumeName3,Files_DirectoryName3]),
    level=4)
pprint(dirList)

# Copy a file in the SciServer file system

In [ ]:
# upload a text string into a file in the remote directory
Files_FileServiceName = "FileServiceJHU"

Files_RootVolumeNameNow = "Storage"
Files_VolumeNameNow = "persistent"
Files_DirectoryNameNow = ""

Files_RootVolumeMoveTo = "Storage"
Files_VolumeNameMoveTo = "created_through_example_notebook"
Files_DirectoryNameMoveTo = ""

Files_RemoteFileName = "sharknado.csv"

fileService = Files.getFileServiceFromName(Files_FileServiceName)


# copy remote file into remote directory

Files.move(fileService, 
"/".join([Files_RootVolumeNameNow,Authentication_loginName,Files_VolumeNameNow,Files_DirectoryNameNow,Files_RemoteFileName]), 
           fileService, 
"/".join([Files_RootVolumeMoveTo,Authentication_loginName,Files_VolumeNameMoveTo,Files_DirectoryNameMoveTo,Files_RemoteFileName])
)

print('File copied')

# Download file

In [ ]:
# download a remote text file into a local directory
Files_FileServiceName = "FileServiceJHU"

Files_RootVolumeToDownloadFrom = "Storage"
Files_VolumeNameToDownloadFrom = "created_through_example_notebook"
Files_DirectoryNameToDownloadFrom = ""
Files_RemoteFileName = "sharknado.csv"
Files_LocalFileName = "sharknado_downloaded.csv"

fileService = Files.getFileServiceFromName(Files_FileServiceName)

Files.download(fileService, 
    "/".join([Files_RootVolumeToDownloadFrom,Authentication_loginName,Files_VolumeNameToDownloadFrom,Files_DirectoryNameToDownloadFrom,Files_RemoteFileName]) 
    ,localFilePath=Files_LocalFileName)

print('Downloaded file, saved as {0:}'.format(Files_LocalFileName))

# Delete file from SciServer file system

In [ ]:
# Delete remote file
Files_FileServiceName = "FileServiceJHU"

Files_RootVolumeToDeleteIn = "Storage"
Files_VolumeNameToDeleteIn = "created_through_example_notebook"
Files_DirectoryNameToDeleteIn = ""
Files_RemoteFileName = "sharknado.csv"

fileService = Files.getFileServiceFromName(Files_FileServiceName)

Files.delete(fileService, 
"/".join([Files_RootVolumeToDeleteIn,Authentication_loginName,Files_VolumeNameToDeleteIn,Files_DirectoryNameToDeleteIn,Files_RemoteFileName]))

print('Deleted file {0:} from volume {1:}'.format(Files_RemoteFileName, Files_VolumeNameToDeleteIn))

# Delete directory from SciServer file system

In [ ]:
# delete folders
# Delete remote file
Files_FileServiceName = "FileServiceJHU"

Files_RootVolumeToDeleteIn = "Storage"
Files_VolumeNameToDeleteIn = "persistent"
Files_DirectoryNameToDelete = "directory_created_through_example_notebook"

fileService = Files.getFileServiceFromName(Files_FileServiceName)


Files.delete(fileService, 
    "/".join([Files_RootVolumeToDeleteIn,Authentication_loginName,Files_VolumeNameToDeleteIn,Files_DirectoryNameToDelete]))

print('Deleted folder {0:} from volume {1:}'.format(Files_DirectoryNameToDelete, Files_VolumeNameToDeleteIn))

# Delete a user volume

In [ ]:
# delete user volumes
Files_FileServiceName = "FileServiceJHU"

Files_RootVolumeToDeleteIn = "Storage"
Files_VolumeNameToDelete = "created_through_example_notebook"

fileService = Files.getFileServiceFromName(Files_FileServiceName)


Files.deleteUserVolume(fileService, 
    "/".join([Files_RootVolumeToDeleteIn, Authentication_loginName, Files_VolumeNameToDelete]))

print('Deleted volume {0:}'.format(Files_VolumeNameToDelete))